# Теоретические вопросы

*1) Какую роль выполняет функция активации в полносвязной нейросети? Также нужно пояснить, почему стакать несколько подряд идущих полносвязных слоев без активации не является хорошей практикой.* **0.5 балла**

*2) Допустим у вас есть MLP ("multi layer perceptron" или полносвязная нейросеть) с 10-ю входными нейронами, за ними 50 нейронов в скрытом слое, на выходе 3 нейрона. Все нейроны снабжены функцией активации ReLU.* **1 балл**
- Какова размерность матрицы $X$ на входе, если обучение проводится батчами. Размер батча $batch\_size$
- Какова размерность матрицы весов скрытого слоя $W_h$ и какова размерность вектора байеса $b_h$ этого скрытого слоя?
- Какова размерность матрицы весов выходного слоя $W_o$ и какова размерность вектора байеса $b_o$ выходного слоя?
- Какова размерность выходной матрицы $Y$?
- Напишите как выглядит функция, которая считает выходную матрицу $Y$ как функцию от $X$, $W_h$, $W_o$, $b_h$, $b_o$.

*3) Как устроено автоматическое дифференцирование в Tensorflow?* **0.5 балла**
- Используются ли символьные вычисления?
- Используется ли численное дифференцирование?
- Сколько проходов по графу вычислений необходимо совершить,
  чтобы посчитать градиенты для каждого параметра в Tensorflow?
  
*4) Что такое "backpropagation" и для чего он нужен. Чем он отличается от алгоритма градиентного спуска?* **0.5 балла**

*5) Расскажите в чем отличия между mini-batch, batch, stochastic gradient descent. Какой из этих алгоритмов самый популярный и почему.* **0.5 балла**

*6) Можно ли инициализировать первоначально все параметры обучаемой модели MLP единицами? А нулями? Поясните ваш ответ. Можно ли инициализировать нулями параметры $b_o$, $b_h$ из задачи №2?* **0.5 балла**

*7) Вывести лосс-функцию кросс-энтропии из правдоподобия и из теории информации.* **0.5 балла**

# Теоретические ответы

1)Функция активации добавляет нелинейность в модель, делает общую функцию сети нелинейной.<br>
Если стакнуть несколько слоев без активации, то модель останется линейной, просто будет больше параметров у <br>модели, но она не сможет работать для сложных данных.<br>
<br>
2)<br>
-$X$ =  [10,batch_size]<br>
-$W_h$ = [49,10] ,  $b_h$ = [3] размерность вектора весов, сам нейрон смещения равен 1<br>
-$W_o$ = [3,49] ,  $b_o$ = в выходном слое не может быть нейрона смещения<br>
-$Y$ =  [3,batch_size]<br>
-$Y$  = $W_o$$*$$W_h$$*$$X$ + $b_h$<br>
<br>
3)<br>
-символьные вычисления = нет<br>
-численное дифференцирование = нет<br>
-один проход<br>
<br>
4)backpropagation - это метод обратного распространения ошибки в графе вычислений нейросети.<br>
На выходе сети считается градиент ошибки и дальше с помощью вычисления частных произовдных в узлах <br>
графа вычислений градиент проходит глубже и для каждого веса определяется градиент на который его нужно скорректировать<br>
Градиентный спуск позволяет итеративно уменьшать ошибку путем коррекции весов ф-ии на величину градиента в сторону<br> его убывания. В нейросетях используется для нахождения изначального градиента ошибки на выходе нейросети<br><br>
5)batch = градиент вычисляется для всего набора данных и обновляет веса только 1 раз за 1 проход по всем данным, зато самый точный<br>
mini-batch = градиент считается для небольшой части данных и каждый раз обновляет веса <br>
stochastic = считает градиент для каждого экземпляра данных и каждый раз обновляет веса <br>
stochastic используется чаще всего, т.к. производит обучение быстрее остальных вариантов, при этом при небольшом <br>learning rate обеспечивает хорошую сходимость <br><br>
6) Можно инициализировать и 1 и 0 , но в таком случае все веса в слое будут изменяться одинаково и <br>
модель получится слабая. Обычно веса инициализируют случайными значениями из определенных диапазонов, но <br>
разными значениями.<br>
параметры $b_o$, $b_h$  нельзя инициализировать нулями, так как нейроны смещения всегда равны 1<br>
7)<br>
<br>

# Первые шаги с TensorFlow

[Описание данных](https://developers.google.com/machine-learning/crash-course/california-housing-data-description)

In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

Подгружаем данные

In [3]:
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index)
)

california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
11437,-114.3,34.2,15.0,5612.0,1283.0,1015.0,472.0,1.5,66.9
5338,-114.5,34.4,19.0,7650.0,1901.0,1129.0,463.0,1.8,80.1
7223,-114.6,33.7,17.0,720.0,174.0,333.0,117.0,1.7,85.7
16194,-114.6,33.6,14.0,1501.0,337.0,515.0,226.0,3.2,73.4
1288,-114.6,33.6,20.0,1454.0,326.0,624.0,262.0,1.9,65.5


## EDA

Посмотрим на данные

In [3]:
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207.3
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,116.0
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,15.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119.4
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180.4
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500.0


## Первая модель

В этом задании требуется предсказать стоимовть жилья (`median_house_value`).

Мы будем делать это с помощью [LinearRegressor](https://www.tensorflow.org/api_docs/python/tf/estimator/LinearRegressor) и [tf estimators](https://www.tensorflow.org/get_started/estimator).

### 1. Определяем target

In [4]:
targets = california_housing_dataframe["median_house_value"]

### 2. Определяем входы в модель

С помощью [Dataset API](https://www.tensorflow.org/programmers_guide/datasets) и [Feature columns](https://www.tensorflow.org/guide/feature_columns) мы с легкостью определяем, как данные будут подаваться в модель.

In [5]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    
    # ключ total_rooms присутствует в features
    features = {key: np.array(value) for key, value in dict(features).items()}                                           
 
    ds = tf.data.Dataset.from_tensor_slices((features, targets))
    # будем выдавать данные батчами в течение num_epochs эпох
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    if shuffle:
        # делаем шафл только во время обучения
        # сможете ответить, зачем нужен шафл данных?
        ds = ds.shuffle(buffer_size=10000)

    return ds

my_feature = california_housing_dataframe[["total_rooms"]]
# по ключу "total_rooms" feature_column будет вытаскивать из tf.data.Dataset нужные фичи
feature_columns = [tf.feature_column.numeric_column("total_rooms", shape=(1,))]

In [8]:
{key: np.array(value) for key, value in dict(my_feature).items()}

{'total_rooms': array([5612., 7650.,  720., ..., 2677., 2672., 1820.])}

In [2]:
tf.enable_eager_execution()

In [9]:
for i in my_input_fn(california_housing_dataframe[['total_rooms']],targets):
    print(i)
    break

({'total_rooms': <tf.Tensor: id=751697, shape=(1,), dtype=float64, numpy=array([2587.])>}, <tf.Tensor: id=751698, shape=(1,), dtype=float64, numpy=array([88.6])>)


### 3. Определяем Модель

In [6]:
lr = 0.00001
my_optimizer = tf.train.AdamOptimizer(learning_rate=lr)

# посмотрите, какие аргументы принимает LinearRegressor
linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=feature_columns,
    optimizer=my_optimizer
)

### 4. Обучаем модель

In [7]:
_ = linear_regressor.train(
    input_fn = lambda: my_input_fn(my_feature, targets),
    steps=100  # 1 step -- это проход по одному батчу
)

### 5. Считаем качество

In [9]:
prediction_input_fn = lambda: my_input_fn(my_feature, targets, num_epochs=1, shuffle=False)
# питоновский генератор
predictions = linear_regressor.predict(input_fn=prediction_input_fn)

# переводим в numpy
predictions = np.array([item['predictions'][0] for item in predictions])

# считаем метрики
mean_squared_error = metrics.mean_squared_error(predictions, targets)
root_mean_squared_error = np.sqrt(mean_squared_error)

print("Mean Squared Error (on training data): %0.3f" % mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % root_mean_squared_error)

Mean Squared Error (on training data): 55522.875
Root Mean Squared Error (on training data): 235.633


## Перебор гиперпараметров

In [10]:
def train_model(learning_rate, num_epochs, batch_size, input_feature="total_rooms"):

    my_feature_data = california_housing_dataframe[[input_feature]]
    targets = california_housing_dataframe["median_house_value"]

    # Create feature columns.
    feature_columns = [tf.feature_column.numeric_column(input_feature, shape=(1,))]

    # Create input functions.
    training_input_fn = lambda: my_input_fn(my_feature_data, targets, batch_size=batch_size, num_epochs=num_epochs)
    prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)

    # Create a linear regressor object.
    my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

    linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=feature_columns,
      optimizer=my_optimizer
    )

    print("Training model...")
    linear_regressor.train(input_fn=training_input_fn)

    predictions = linear_regressor.predict(input_fn=prediction_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])

    root_mean_squared_error = np.sqrt(metrics.mean_squared_error(predictions, targets))

    print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)

## Задание 1 (2 балла)

Попробуйте поиграть с гиперпараметрами и достигнуть $RMSE \leq 180$

Также можно попробовать:
1. Другой оптимайзер ([здесь](https://www.tensorflow.org/api_docs/python/tf/train) можно посмотреть, какие бывают)
2. Комбинацию фичей (**feature_columns** -- это list, в котором несколько tf.feature_column.numeric_column)
3. Поиграть с аргументами модели (посмотрите, какие аргументы есть у LinearRegressor)

p.s.
Попробуйте переписать функцию `train_model`, чтобы было удобней изменять параметры модели.

In [16]:
# пример обучения модели
train_model(
    learning_rate=0.0001,
    num_epochs=2,
    batch_size=10
)

Training model...
Final RMSE (on training data): 169.49


In [21]:
def train_model(learning_rate, num_epochs, batch_size, input_feature="total_rooms"):

    my_feature_data = california_housing_dataframe[[input_feature]]
    targets = california_housing_dataframe["median_house_value"]

    # Create feature columns.
    feature_columns = [tf.feature_column.numeric_column(input_feature, shape=(1,))]

    # Create input functions.
    training_input_fn = lambda: my_input_fn(my_feature_data, targets, batch_size=batch_size, num_epochs=num_epochs)
    prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)

    # Create a linear regressor object.
    my_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

    linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=feature_columns,
      optimizer=my_optimizer
    )

    print("Training model...")
    linear_regressor.train(input_fn=training_input_fn)

    predictions = linear_regressor.predict(input_fn=prediction_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])

    root_mean_squared_error = np.sqrt(metrics.mean_squared_error(predictions, targets))

    print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)

In [22]:
train_model(
    learning_rate=0.00001,
    num_epochs=3,
    batch_size=10
)

Training model...
Final RMSE (on training data): 168.43


## Задание 2 (2 балла)

Попробуйте подать в модель другие фичи и достигнуть $RMSE \leq 170$

In [32]:
def train_model(learning_rate, num_epochs, batch_size, input_features=["total_rooms"]):

    my_feature_data = california_housing_dataframe[input_features]
    targets = california_housing_dataframe["median_house_value"]

    # Create feature columns.
    
    feature_columns = [tf.feature_column.numeric_column(input_feature, shape=(1,)) for input_feature in input_features]

    # Create input functions.
    training_input_fn = lambda: my_input_fn(my_feature_data, targets, batch_size=batch_size, num_epochs=num_epochs)
    prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)

    # Create a linear regressor object.
    my_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

    linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=feature_columns,
      optimizer=my_optimizer
    )

    print("Training model...")
    linear_regressor.train(input_fn=training_input_fn)

    predictions = linear_regressor.predict(input_fn=prediction_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])

    root_mean_squared_error = np.sqrt(metrics.mean_squared_error(predictions, targets))

    print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)

In [31]:
train_model(
    learning_rate=0.0001,
    num_epochs=3,
    batch_size=10,
    input_features = ['housing_median_age','median_income','households']
)

Training model...
Final RMSE (on training data): 160.48


## Задание 3 (2 балла)

Вместо `tf.estimator.LinearRegressor` попробуйте применить другую модель и достигнуть $RMSE \leq 160$

In [40]:
def train_model(learning_rate, num_epochs, batch_size, input_features=["total_rooms"]):

    my_feature_data = california_housing_dataframe[input_features]
    targets = california_housing_dataframe["median_house_value"]

    # Create feature columns.
    
    feature_columns = [tf.feature_column.numeric_column(input_feature, shape=(1,)) for input_feature in input_features]

    # Create input functions.
    training_input_fn = lambda: my_input_fn(my_feature_data, targets, batch_size=batch_size, num_epochs=num_epochs)
    prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)

    # Create a linear regressor object.
    my_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

    linear_regressor = tf.estimator.DNNRegressor(
      feature_columns=feature_columns,
      optimizer=my_optimizer,
      hidden_units=[3]
    )

    print("Training model...")
    linear_regressor.train(input_fn=training_input_fn)

    predictions = linear_regressor.predict(input_fn=prediction_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])

    root_mean_squared_error = np.sqrt(metrics.mean_squared_error(predictions, targets))

    print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)

In [42]:
train_model(
    learning_rate=0.0001,
    num_epochs=20,
    batch_size=10,
    input_features = ['housing_median_age','median_income','households']
)

Training model...
Final RMSE (on training data): 118.42
